In [1]:
@file:DependsOn("com.google.adk:google-adk:0.3.0")
@file:DependsOn("com.google.adk:google-adk-dev:0.2.0")
@file:DependsOn("com.google.genai:google-genai:1.23.0")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.8.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-rx3:1.8.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-reactive:1.8.1")

In [2]:
import com.google.adk.agents.LlmAgent

import com.google.adk.models.Gemini
import com.google.genai.Client

import com.google.adk.runner.InMemoryRunner
import com.google.adk.sessions.Session
import com.google.adk.sessions.InMemorySessionService
import com.google.adk.tools.BaseTool
import com.google.adk.tools.mcp.McpTool
import com.google.adk.tools.mcp.McpToolset
import com.google.adk.tools.ToolContext
import com.google.adk.tools.mcp.McpSessionManager
import com.google.adk.tools.mcp.StdioServerParameters
import com.google.adk.tools.mcp.SseServerParameters
import com.google.adk.agents.ReadonlyContext

import com.google.genai.types.Content
import com.google.genai.types.Part

import java.util.UUID
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.toList
import kotlinx.coroutines.reactive.asFlow

import io.reactivex.rxjava3.core.Flowable;

import io.modelcontextprotocol.client.transport.ServerParameters
import java.util.Optional

# 🚀 Agent Tool Patterns and Best Practices

**Welcome to Day-2 of the Kaggle 5-day Agents course!**

        In the previous notebook, you learned how to add custom Python functions as tools to your agent. In this notebook, we'll take the next step: **consuming external MCP services** and handling **long-running operations**.

In this notebook, you'll learn how to:

- ✅ **Connect to external MCP servers**
        - ✅ **Implement long-running operations** that can pause agent execution for external input
        - ✅ **Build resumable workflows** that maintain state across conversation breaks
        - ✅ Understand when and how to use these patterns

In [4]:
val apiKey = System.getenv("GOOGLE_API_KEY")
if (apiKey == null) {
    throw IllegalStateException("Please set the GOOGLE_API_KEY environment variable.")
}
val githubKey = System.getenv("GITHUB_TOKEN")
if (githubKey == null) {
    throw IllegalStateException("Please set the GITHUB_TOKEN environment variable.")
}

In [5]:
val geminiClient = Client.builder().apiKey(apiKey).build()

In [6]:
val geminiModel = Gemini(
    "gemini-2.5-flash-lite", // Your model name
    geminiClient
)

---
## 🧰 Section 2: Model Context Protocol

So far, you have learned how to create custom functions for your agents. But connecting to external systems (GitHub, databases, Slack) requires writing and maintaining API clients.

**Model Context Protocol (MCP)** is an open standard that lets agents use community-built integrations. Instead of writing your own integrations and API clients, just connect to an existing MCP server.

MCP enables agents to:

✅ **Access live, external data** from databases, APIs, and services without custom integration code
✅ **Leverage community-built tools** with standardized interfaces
✅ **Scale capabilities** by connecting to multiple specialized servers

### 2.1: How MCP Works

MCP connects your agent (the **client**) to external **MCP servers** that provide tools:

- **MCP Server**: Provides specific tools (like image generation, database access)
- **MCP Client**: Your agent that uses those tools
- **All servers work the same way** - standardized interface

**Architecture:**
```
┌──────────────────┐
│   Your Agent     │
│   (MCP Client)   │
└────────┬─────────┘
│
│ Standard MCP Protocol
│
┌────┴────┬────────┬────────┐
│         │        │        │
▼         ▼        ▼        ▼
┌────────┐ ┌─────┐ ┌──────┐ ┌─────┐
│ GitHub │ │Slack│ │ Maps │ │ ... │
│ Server │ │ MCP │ │ MCP  │ │     │
└────────┘ └─────┘ └──────┘ └─────┘
```

### 2.2: Using MCP with Your Agent

The workflow is simple:

1. Choose an MCP Server and tool
2. Create the MCP Toolset (configure connection)
3. Add it to your agent
4. Run and test the agent

**Step 1: Choose MCP Server**

For this demo, we'll use the **[Everything MCP Server](https://github.com/modelcontextprotocol/servers/tree/main/src/everything)** - an npm package (`@modelcontextprotocol/server-everything`) designed for testing MCP integrations.

It provides a `getTinyImage` tool that returns a simple test image (16x16 pixels, Base64-encoded). **Find more servers:** [modelcontextprotocol.io/examples](https://modelcontextprotocol.io/examples)

**‼️ NOTE: This is a demo server to learn MCP.** In production, you'll use servers for Google Maps, Slack, Discord, etc.

**Step 2: Create the MCP Toolset**

The `McpToolset` is used to integrate an ADK Agent with an MCP Server.

**What the code does:**
- Uses `npx` (Node package runner) to run the MCP server
- Connects to `@modelcontextprotocol/server-everything`
- Filters to only use the `getTinyImage` tool (the server has others, but we only need this one)

In [7]:
val srvParams = StdioServerParameters.builder()
    .command("npx") // Run MCP server via npx
    .args(listOf("-y", "@modelcontextprotocol/server-everything"))
    .build().toServerParameters()

val mcpImageServer = McpToolset(
    srvParams,
    // If no filtering provided, these will be the tools available:
    // echo -> works
    // add -> works
    // annotatedMessage -> fails the include image flag
    // longRunningOperation
    // printEnv -> DON'T RUN WILL LEAK GOOGLE_API_KEY
    // sampleLLM
    // getTinyImage -> returns text data but not the Base64 image
    // getResourceReference -> works
    // getResourceLinks -> can't return resource links
    // structuredContent
    // These tools work fine on the Python client but somehow not on Java/Kotlin
    Optional.ofNullable(listOf("getResourceLinks"))
)

println("✅ MCP Tool created")
val tools = mcpImageServer.getTools(ReadonlyContext(null))
tools.blockingForEach {
    println(it.name())
}

✅ MCP Tool created
getResourceLinks


In [8]:
val mcpAgent = LlmAgent.builder()
    .name("mcp_agent")
    .model(geminiModel)
    // Updated instruction
    .description("An agent that retrieves data from MCP")
    .instruction("""
        Use the MCP Tool to generate perform the user queries
    """)
    .tools(
        mcpImageServer
    )
    .build()

println("✅ MCP agent created")

✅ MCP agent created


In [9]:
var runner: InMemoryRunner = InMemoryRunner(mcpAgent, "MCP Flow")

In [10]:
val prompt = "Please give some resource links"
var userMsg: Content? = Content.fromParts(Part.fromText(prompt))

In [11]:
var session: Session? = runner
    .sessionService()
    .createSession("MCP Flow", "user")
    .blockingGet()

In [12]:
println("Agent response ->")
var resultText = ""
runBlocking {
    var events = runner.runAsync("user", session.id(), userMsg)
        .asFlow()
        .toList()

    events.forEach {
        println("----")
        println(it.stringifyContent())
    }
}

Agent response ->
----
Function Call: FunctionCall{id=Optional[adk-c966166a-9fb3-4aab-b9ba-fdbe869a085f], args=Optional[{count=3}], name=Optional[getResourceLinks]}
----
Function Response: FunctionResponse{willContinue=Optional.empty, scheduling=Optional.empty, parts=Optional.empty, id=Optional[adk-c966166a-9fb3-4aab-b9ba-fdbe869a085f], name=Optional[getResourceLinks], response=Optional[{text_output=[{text=Here are 3 resource links to resources available in this server (see full output in tool response if your client does not support resource_link yet):}]}]}
----
I have provided 3 resource links for you. Please let me know if you require any more assistance.


In [13]:
var events = runner.runAsync("user", session.id(), userMsg)
println("Agent response ->")
events.blockingForEach { event -> println(event.stringifyContent()) }

Agent response ->
Function Call: FunctionCall{id=Optional[adk-98002ea1-dfa9-4a5e-b38a-f52be11eb787], args=Optional[{count=3}], name=Optional[getResourceLinks]}
Function Response: FunctionResponse{willContinue=Optional.empty, scheduling=Optional.empty, parts=Optional.empty, id=Optional[adk-98002ea1-dfa9-4a5e-b38a-f52be11eb787], name=Optional[getResourceLinks], response=Optional[{text_output=[{text=Here are 3 resource links to resources available in this server (see full output in tool response if your client does not support resource_link yet):}]}]}
I have provided 3 resource links for you. Please let me know if you need more.


### 2.3: Extending to Other MCP Servers

The same pattern works for any MCP server - only the `connection_params` change. Here are some examples:

#### **👉 Kaggle MCP Server** - For dataset and notebook operations

Kaggle provides an MCP server that lets your agents interact with Kaggle datasets, notebooks, and competitions.

**Connection example:**
```python
McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command='npx',
            args=[
                '-y',
                'mcp-remote',
                'https://www.kaggle.com/mcp'
            ],
        ),
        timeout=30,
    )
)
```

**What it provides:**
- 📊 Search and download Kaggle datasets
        - 📓 Access notebook metadata
        - 🏆 Query competition information etc.,

**Learn more:** [Kaggle MCP Documentation](https://www.kaggle.com/docs/mcp)

#### **👉 GitHub MCP Server** - For PR/Issue analysis

```python
McpToolset(
    connection_params=StreamableHTTPServerParams(
        url="https://api.githubcopilot.com/mcp/",
        headers={
            "Authorization": f"Bearer {GITHUB_TOKEN}",
            "X-MCP-Toolsets": "all",
            "X-MCP-Readonly": "true"
        },
    ),
)
```

**More resources:** [ADK Third-party Tools Documentation](https://google.github.io/adk-docs/tools/third-party/)

In [284]:
val kaggleSrvParams = StdioServerParameters.builder()
    .command("npx") // Run MCP server via npx
    .args(listOf("-y", "mcp-remote", "https://www.kaggle.com/mcp"))
    .build().toServerParameters()

val mcpKaggleServer = McpToolset(
    kaggleSrvParams
)

println("✅ Kaggle MCP Tool created")
val kaggleTools = mcpKaggleServer.getTools(ReadonlyContext(null))
kaggleTools.blockingForEach {
    println(it.name())
}

✅ Kaggle MCP Tool created
get_competition_data_files_summary
get_competition_leaderboard
update_dataset_metadata
download_dataset
download_competition_leaderboard
get_model_variation
get_dataset_files_summary
list_model_variation_versions
get_benchmark_leaderboard
download_model_variation_version
get_notebook_info
create_code_competition_submission
list_models
create_model
get_dataset_status
create_benchmark_task_from_prompt
get_notebook_session_status
download_notebook_output_zip
update_model
get_model
download_competition_data_file
get_dataset_info
list_model_variation_version_files
get_competition_submission
cancel_notebook_session
authorize
list_competition_data_files
search_competition_submissions
get_dataset_metadata
submit_to_competition
download_notebook_output
list_dataset_files
get_competition
list_model_variations
list_competition_data_tree_files
update_model_variation
download_competition_data_files
search_competitions
save_notebook
list_notebook_files
search_notebooks
star

In [14]:
import com.google.adk.tools.mcp.StreamableHttpServerParameters
import java.time.Duration

val githubSrvParams = StreamableHttpServerParameters(
        "https://api.githubcopilot.com/mcp/",
        mapOf(
            "Authorization" to "Bearer $githubKey",
            "X-MCP-Toolsets" to "all",
            "X-MCP-Readonly" to "true"
        ),
        Duration.ofSeconds(30L),
        Duration.ofSeconds(30L),
        true
)

val mcpGithubServer = McpToolset(
    githubSrvParams
)

println("✅ Github MCP Tool created")
val githubTools = mcpGithubServer.getTools(ReadonlyContext(null))
githubTools.blockingForEach {
    println(it.name())
}

✅ Github MCP Tool created
download_workflow_run_artifact
get_code_scanning_alert
get_commit
get_copilot_space
get_dependabot_alert
get_discussion
get_discussion_comments
get_file_contents
get_gist
get_global_security_advisory
get_job_logs
get_label
get_latest_release
get_me
get_notification_details
get_project
get_project_field
get_project_item
get_release_by_tag
get_secret_scanning_alert
get_tag
get_team_members
get_teams
get_workflow_run
get_workflow_run_logs
get_workflow_run_usage
github_support_docs_search
issue_read
list_branches
list_code_scanning_alerts
list_commits
list_copilot_spaces
list_dependabot_alerts
list_discussion_categories
list_discussions
list_gists
list_global_security_advisories
list_issue_types
list_issues
list_label
list_notifications
list_org_repository_security_advisories
list_project_fields
list_project_items
list_projects
list_pull_requests
list_releases
list_repository_security_advisories
list_secret_scanning_alerts
list_starred_repositories
list_tags
list_

In [15]:
val githubMcpAgent = LlmAgent.builder()
    .name("github_mcp_agent")
    .model(geminiModel)
    // Updated instruction
    .description("An agent that retrieves data from Github MCP")
    .instruction("""
        Use the MCP Tool to generate perform the user queries
    """)
    .tools(
        mcpGithubServer
    )
    .build()

println("✅ Github MCP agent created")

✅ Github MCP agent created
